In [1]:
import pandas as pd
import regex as re
import string
import json
import csv

### Document ID and Answer Text

In [2]:
collection = pd.read_csv("train/FiQA_train_doc_final.tsv", sep="\t")
collection = collection[['docid', 'doc']]
collection = collection.sort_values(by=['docid'])

collection.head(5)

,docid,doc
0,3,I'm not saying I don't like the idea of on-the...
1,31,So nothing preventing false ratings besides ad...
2,56,You can never use a health FSA for individual ...
3,59,Samsung created the LCD and other flat screen ...
4,63,Here are the SEC requirements: The federal sec...


In [15]:
print("Example passage: \n")
print(collection.iloc[0]['doc'])

Example passage: 

I'm not saying I don't like the idea of on-the-job training too, but you can't expect the company to do that. Training workers is not their job - they're building software. Perhaps educational systems in the U.S. (or their students) should worry a little about getting marketable skills in exchange for their massive investment in education, rather than getting out with thousands in student debt and then complaining that they aren't qualified to do anything.


In [3]:
print("Total number of passages: {}".format(len(collection)))

Total number of passages: 57638


In [25]:
# Cleaning data
empty_docs = []
empty_id = []

for index, row in collection.iterrows():
    if pd.isna(row['doc']):
        empty_docs.append(row['docid'])
        empty_id.append(index)

# print(empty_docs)
print(empty_id)

# Remove empty answers
collection2 = collection.drop(empty_id)

collection2.head(5)

[741, 1158, 1329, 3208, 3937, 5266, 6399, 11237, 11407, 12328, 13459, 14869, 16143, 16326, 18297, 19839, 19944, 20739, 22923, 23949, 24603, 28093, 30966, 31492, 34386, 34539, 34697, 38382, 39802, 42403, 43008, 45178, 46794, 50845, 53111, 54963, 56460, 57452]


,docid,doc
0,3,I'm not saying I don't like the idea of on-the...
1,31,So nothing preventing false ratings besides ad...
2,56,You can never use a health FSA for individual ...
3,59,Samsung created the LCD and other flat screen ...
4,63,Here are the SEC requirements: The federal sec...


In [26]:
len(collection2)

57600

In [27]:
collection2.tail(10)

,docid,doc
57628,599874,Looks like it's $500 to start (certificate of ...
57629,599876,You are in business for yourself. You file Sch...
57630,599898,"Well, I'm not an expert and you sound pretty c..."
57631,599925,Investopedia has a good explanation of the ter...
57632,599939,"&gt;Yeah, that's what I meant when I said 'fel..."
57633,599946,"&gt;Well, first off, the roads are more than j..."
57634,599953,Yes they do. There are billions and billions s...
57635,599966,&gt;It's biggly sad you don't understand human...
57636,599975,"Did your CTO let a major group use ""admin/admi..."
57637,599987,Giving the government more control over the di...


In [29]:
# Write collection df to file
with open("collection_new.tsv",'w') as write_tsv:
    write_tsv.write(collection2.to_csv(sep='\t', index=False, header=False))

In [42]:
# Convert collection df to JSON file for Anserini's document indexer
output_jsonl_file = open("docs00.json", 'w', encoding='utf-8', newline='\n')

with open("collection_new.tsv", encoding='utf-8') as f:
    for i, line in enumerate(f):
        id_text = line.rstrip().split('\t')
        doc_id, doc_text = id_text[0], id_text[1]
        
        output_dict = {'id': doc_id, 'contents': doc_text}
        
        output_jsonl_file.write(json.dumps(output_dict) + '\n')

In [ ]:
with open("foobar.json", "w") as json_file:
    json.dump(json_data, json_file, indent=4)
    json_file.write("\n") 

### Question ID and Question Text

In [18]:
# Question ID and Question text pair
query_df = pd.read_csv("train/FiQA_train_question_final.tsv", sep="\t")

queries = query_df[['qid', 'question']]

queries.head(5)

,qid,question
0,0,What is considered a business expense on a bus...
1,1,Claiming business expenses for a business with...
2,2,Transferring money from One business checking ...
3,3,Having a separate bank account for business/in...
4,4,Business Expense - Car Insurance Deductible Fo...


In [19]:
print("Example Question: \n")
print(queries.iloc[0]['question'])

Example Question: 

What is considered a business expense on a business trip?


In [22]:
print("Number of questions: {}".format(len(queries)))

Number of questions: 6648


In [ ]:
# Write queries to file
with open("fiqa-passage/queries_train.tsv",'w') as write_tsv:
    write_tsv.write(queries.to_csv(sep='\t', index=False, header=False))

### Question ID and Answer ID pair

In [24]:
# Question ID and Answer ID pair
qid_docid = pd.read_csv("train/FiQA_train_question_doc_final.tsv", sep="\t")

qid_docid = qid_docid [['qid', 'docid']]

qid_docid.head(5)
# qid_docid = qid_docid.assign(new1=0)
# qid_docid = qid_docid.assign(new2=1)
# qrels_train = qid_docid[['qid', 'new1', 'docid', 'new2']]

,qid,docid
0,0,18850
1,1,14255
2,2,308938
3,3,296717
4,3,100764


In [30]:
print("Number of Question Answer Pairs: {} \n".format(len(qid_docid)))

Number of Question Answer Pairs: 17110 



In [31]:
# with open("fiqa-passage/qrels_train.tsv",'w') as write_tsv:
#     write_tsv.write(qrels_train.to_csv(sep='\t', index=False, header=False))
# rel = qid_docid['qid'] == 0

# rel_df = qid_docid[rel]

### Example QA pair

In [39]:
print(queries.at[3, 'question'])

x = collection[collection['docid']==296717]

print()
print(x.at[28732, 'doc'])

Having a separate bank account for business/investing, but not a “business account?”

Having a separate checking account for the business makes sense. It simplifies documenting your income/expenses. You can "explain" every dollar entering and exiting the account without having to remember that some of them were for non-business items. My credit union allowed me to have a 2nd checking account and allowed me to put whatever I wanted as the name on the check. I think this looked a little better than having my name on the check. I don't see the need for a separate checking account for investing. The money can be kept in a separate savings account that has no fees, and can even earn a little interest. Unless you are doing a lot of investment transactions a month this has worked for me. I fund IRAs and 529 plans this way. We get paychecks 4-5 times a month, but send money to each of the funds once a month. You will need a business account if the number of transactions becomes large. If you d